In [2]:
"""
Linking aggregated FRP to climate and topography
Author: maxwell.cook@colorado.edu
"""

import os, sys, gc, time
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.stats import pearsonr

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [3]:
fp = os.path.join(projdir,'data/tabular/mod/viirs_snpp_jpss1_gridstats_fortypcd.csv')
tm = pd.read_csv(fp)
tm.drop(columns=['Unnamed: 0'], inplace=True)
tm.columns

Index(['grid_index', 'grid_area', 'afd_count', 'unique_days', 'overlap',
       'frp_csum', 'frp_max', 'frp_min', 'frp_mean', 'frp_p90', 'frp_first',
       'day_max_frp', 'dt_max_frp', 'first_obs_date', 'last_obs_date',
       't4_max', 't4_mean', 't5_max', 't5_mean', 'day_count', 'night_count',
       'frp_max_day', 'frp_max_night', 'frp_mean_day', 'frp_mean_night',
       'frp_p90_day', 'frp_p90_night', 'frp_first_day', 'frp_first_night',
       'Fire_ID', 'Fire_Name', 'geometry', 'grid_x', 'grid_y', 'SpeciesName',
       'spp_pct', 'forest_pct', 'count', 'total_pixels'],
      dtype='object')

In [4]:
# Load the topo summary
fp = os.path.join(projdir,'data/earth-engine/exports/topo/gridstats_topo.csv')
topo = pd.read_csv(fp)
topo.columns

Index(['system:index', 'chili', 'elev', 'grid_index', 'slope', 'tpi', '.geo'], dtype='object')

In [18]:
# Load the gridmet summary
fp = os.path.join(projdir,'data/earth-engine/exports/gridmet/gridstats_gridmet.csv')
climate = pd.read_csv(fp)
climate.columns

Index(['system:index', 'Fire_ID', 'erc', 'erc_dv', 'first_obs', 'vpd',
       'vpd_dv', '.geo'],
      dtype='object')

In [19]:
# Join to the grid by Fire_ID and first_obs
print(len(tm))
# tidy the columns
climate = climate[['Fire_ID', 'first_obs', 'erc', 'erc_dv', 'vpd', 'vpd_dv']]
climate.rename(columns={'first_obs': 'first_obs_date'}, inplace=True)
tm['first_obs_date'] = tm['first_obs_date'].astype(str) # to match gee output
# do the join
grid_clim = tm.merge(climate, on=['Fire_ID', 'first_obs_date'], how='left')
print(len(grid_clim))

grid_clim[['Fire_ID', 'grid_index', 'SpeciesName', 'first_obs_date', 'erc', 'erc_dv', 'vpd', 'vpd_dv']].head()

153375
153375


,Fire_ID,grid_index,SpeciesName,first_obs_date,erc,erc_dv,vpd,vpd_dv
0,14,824824,Aspen,2018-07-05,64.997051,3.103695,1.547844,0.34404
1,14,824824,Douglas-fir,2018-07-05,64.997051,3.103695,1.547844,0.34404
2,14,824824,Piñon-juniper,2018-07-05,64.997051,3.103695,1.547844,0.34404
3,14,824824,Ponderosa,2018-07-05,64.997051,3.103695,1.547844,0.34404
4,14,824824,Spruce-fir,2018-07-05,64.997051,3.103695,1.547844,0.34404


In [22]:
# Merge the topo and climate with the frp/forest grid
topo = topo[['grid_index', 'elev', 'slope', 'chili', 'tpi']]
grid_clim_topo = grid_clim.merge(topo, on='grid_index', how='left')
grid_clim_topo[['grid_index', 'SpeciesName', 'elev', 'slope', 'chili', 'tpi', 'erc']].head(10)

,grid_index,SpeciesName,elev,slope,chili,tpi,erc
0,824824,Aspen,2862.870738,26.369393,217.572440,-56.769463,64.997051
1,824824,Douglas-fir,2862.870738,26.369393,217.572440,-56.769463,64.997051
2,824824,Piñon-juniper,2862.870738,26.369393,217.572440,-56.769463,64.997051
3,824824,Ponderosa,2862.870738,26.369393,217.572440,-56.769463,64.997051
4,824824,Spruce-fir,2862.870738,26.369393,217.572440,-56.769463,64.997051
5,824825,Aspen,2963.771770,18.614423,211.276919,22.066350,64.997051
6,824825,Lodgepole,2963.771770,18.614423,211.276919,22.066350,64.997051
7,824825,Spruce-fir,2963.771770,18.614423,211.276919,22.066350,64.997051
8,824826,Aspen,3020.403483,24.212345,177.530698,59.080197,64.997051
9,824826,Douglas-fir,3020.403483,24.212345,177.530698,59.080197,64.997051


In [23]:
# save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/viirs_snpp_jpss1_gridstats_fortypcd_climtopo.csv')
grid_clim_topo.to_csv(out_fp)
print(f"Saved file to: {out_fp}")

Saved file to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/viirs_snpp_jpss1_gridstats_fortypcd_climtopo.csv


In [ ]:
# plot relationships to FRP

In [ ]:
# topography

cols = ['elev', 'slope', 'chili', 'tpi']

# melt the df to facet plot variables


In [ ]:
# climate / fuel moisture

cols = ['erc', 'erc_dv', 'vpd', 'vpd_dv']

# melt the df to facet plot variables